# 构建模型
参考文献：

[2-3]Channel Estimation for IRS-Assisted Millimeter-Wave MIMO Systems：Sparsity-Inspired Approaches

[2-2]Channel Estimation for RIS-Aided Multiuser Millimeter-Wave Massive MIMO Systems



In [2]:
import numpy as np
import math, cmath
import matplotlib as mpl
import matplotlib.pyplot as plt

## system parameters
BS has a $N$ antannes ULA

RIS has a $M$ antannes ULA

$K$ single antanna users



In [ ]:
class system_parameters():
    def __init__(self, N=100, M=100, K=4, L=5, J=4) -> None:
        self.N = N
        self.M = M
        self.K = K
        self.fc = 28e9  # frequence
        self.lambda_c = 3e8/self.fc
        self.sigma_BR = 2.2 # shadow factor for RIS-BS link 
        self.sigma_RU = 2.8 # shadow factor for RIS-User link

        

$$
\mathbf{a}_{X}(x)=\left[1, e^{-\mathrm{i} 2 \pi x}, \ldots, e^{-\mathrm{i} 2 \pi(X-1) x}\right]^{\mathrm{T}}
$$
where $X \in\{M, N\}$ and $x \in\left\{\omega_{l}, \psi_{l}, \varphi_{k, j}\right\} . \omega_{l}=\frac{d_{\mathrm{RIS}}}{\lambda_{c}} \cos \left(\theta_{l}\right), \psi_{l}=\frac{d_{\mathrm{BS}}}{\lambda_{c}} \cos \left(\phi_{l}\right)$, and $\varphi_{k, j}=$ $\frac{d_{\text {RIS }}}{\lambda_{c}} \cos \left(\vartheta_{k, j}\right)$ are the directional cosine with $\theta_{l}$ and $\phi_{l}$ denoting the AoD and AoA of the $l$-th spatial path from RIS to BS, respectively, and $\vartheta_{k, j}$ as the AoA of the $j$-th spatial path from user $k$ to RIS. $\lambda_{c}$ is the carrier wavelength. 

In [8]:
def ULA_array_response(X, theta, d, lambda_c):
    # Input:
        # X: the dimension of the array, int
        # theta: the AoA/AoD of the in/out signal, float(0~$\pi$)
        # d: array interval
        # lambda_c: carrier wave length

    # Output:
        # alpha: the array response
    alpha = np.array(X*[0+1j])
    for x in range(X):
        phase = 2*math.pi*x*(d/lambda_c)*math.cos(theta)
        alpha[x] = cmath.exp(-1j * phase)
    return alpha

# when f_c = 28GHz, lamda= 3e8 / f_c = 3e8/ 28e9
print("test ULA_array_response")
print(ULA_array_response(8, math.pi/2, 3e8/28e9 /2, 3e8/28e9))
print(ULA_array_response(8, math.pi, 3e8/28e9 /2, 3e8/28e9))
print(ULA_array_response(8, 0, 3e8/28e9 /2, 3e8/28e9))

test ULA_array_response
[1.-0.00000000e+00j 1.-1.92367069e-16j 1.-3.84734139e-16j
 1.-5.77101208e-16j 1.-7.69468277e-16j 1.-9.61835347e-16j
 1.-1.15420242e-15j 1.-1.34656949e-15j]
[ 1.+0.00000000e+00j -1.+1.22464680e-16j  1.-2.44929360e-16j
 -1.+3.67394040e-16j  1.-4.89858720e-16j -1.+6.12323400e-16j
  1.-7.34788079e-16j -1.+8.57252759e-16j]
[ 1.-0.00000000e+00j -1.-1.22464680e-16j  1.+2.44929360e-16j
 -1.-3.67394040e-16j  1.+4.89858720e-16j -1.-6.12323400e-16j
  1.+7.34788079e-16j -1.-8.57252759e-16j]


# channel generation
## generate $\alpha_{l}$
$$
\alpha_{l} \sim \mathcal{C} \mathcal{N}\left(0,10^{-3} d_{\mathrm{BR}}^{-2.2}\right)
$$
## generate $\beta_{k,j} $
$$
\beta_{k, j} \sim \mathcal{C N}\left(0,10^{-3} d_{\mathrm{RU}}^{-2.8}\right)
$$

## generate AoA $\phi_{l}$, AoD $w_l$, AoA $\vartheta_{k, j}$
$$
\omega_{l}, \psi_{l}, \varphi_{k, j}
$$
$$
\left\{\phi_{l}, \theta_{l}, \vartheta_{k, j}\right\} \text { are continuous and uniformly distributed over }[0, \pi)
$$

## the channels:

- $k$-th user to the RIS ${\bf h}_k \in \mathbb{C}^{M \times 1}$
$$
\mathbf{h}_{k}=\mathbf{A}_{M, k} \boldsymbol{\beta}_{k}
$$
- RIS to the BS:    ${\bf H} \in \mathbb{C}^{N \times M}$
$$
\mathbf{H}=\mathbf{A}_{N} \Lambda \mathbf{A}_{M}^{\mathrm{H}}
$$
- cascaded channel $\mathbf{G}_{k}=\operatorname{HDiag}\left(\mathbf{h}_{k}\right) \in \mathbb{C}^{N \times M}$
- angle-domain expression: $\mathbf{G}_{k}=\mathbf{A}_{N} \boldsymbol{\Lambda} \mathbf{A}_{M}^{\mathrm{H}} \operatorname{Diag}\left(\mathbf{A}_{M, k} \boldsymbol{\beta}_{k}\right)$
$$
\begin{aligned}
\mathbf{A}_{N} &=\left[\mathbf{a}_{N}\left(\psi_{1}\right), \ldots, \mathbf{a}_{N}\left(\psi_{L}\right)\right] \in \mathbb{C}^{N \times L} \\
\boldsymbol{\Lambda} &=\operatorname{Diag}\left(\alpha_{1}, \alpha_{2}, \ldots, \alpha_{L}\right) \in \mathbb{C}^{L \times L} \\
\mathbf{A}_{M} &=\left[\mathbf{a}_{M}\left(\omega_{1}\right), \ldots, \mathbf{a}_{M}\left(\omega_{L}\right)\right] \in \mathbb{C}^{M \times L}
\end{aligned}
$$
$$
\begin{aligned}
\mathbf{A}_{M, k} &=\left[\mathbf{a}_{M}\left(\varphi_{k, 1}\right), \ldots, \mathbf{a}_{M}\left(\varphi_{k, J_{k}}\right)\right] \in \mathbb{C}^{M \times J_{k}} \\
\boldsymbol{\beta}_{k} &=\left[\beta_{k, 1}, \ldots, \beta_{k, J_{k}}\right]^{\mathrm{T}} \in \mathbb{C}^{J_{k} \times 1}
\end{aligned}
$$

In [12]:
def userk_to_RIS_channel_generation(A_MK, beta_k):
    # input 
        # A_Mk, array, complex, M times J_k
        # beta_k, array, complex, J_k times 1
    # output
        # h_k, array, complex, M tims 1
    h_k = np.dot(A_MK, beta_k)
    return h_k

def RIS_to_BS_channel_generation(A_N, Lambda, A_M):
    # input
        # A_N, array, complex, N times L
        # Lambda, array, complex, L times L
        # A_M, array, complex, M times L
    # output
        # H, array, complex, N times M
    H = np.dot(np.dot(A_N, Lambda), A_M)
    return H

def cascaded_channel_generation(H, h_k):
    # input：
        # H, array, complex, N times M
        # h_k, array, complex, M tims 1
    # output:
        # G_k, array, complex, N times M
    G_k = np.dot(H, np.diagflat(h_k))
    return G_k

def A_MK_generation(J_k, M, d, lambda_c):
    # input:
        # J_k, int
        # M, int, ant number of the BS
        # d: array interval
        # lambda_c: carrier wave length
    # output:
        # A_Mk, array, complex, M times J_k
        # varphi, array, float(in arc), J_k times 1
    varphi = np.random.rand(J_k,1) * np.pi
    A_Mk = np.array([ULA_array_response(M, angle, d, lambda_c) for angle in varphi])

    return A_Mk, varphi

def beta_k_generation(J_k, d_RU, sigma_RU= 2.8):
    # input:
        # J_k, int
        # d_RU, distance between RIS and BS
        # sigma_RU, mmWave channel path loss fluctuation
    # output:
        # beta_k, array, complex, J_k times 1 
    
    return 0

def A_N_generation():
    return 0

def Lambda_generation():
    return 0

def A_M_generation():
    return 0

def channel_generation(omega, psi, varphi):
    # input:
        # omega, array, float(in arc), L times 1
        # psi, array, float(in arc), L times 1
        # varphi, array, float(in arc), J_k times 1
    # output:
        # G_k, array, complex, N times M

    return 0

In [13]:
A_MK_generation(4,8, 3e8/28e9/2, 3e8/28e9)


array([[ 1.        +0.j        , -0.99995941+0.00901043j,
         0.99983762-0.01802013j, -0.99963467+0.02702837j,
         0.99935055-0.03603441j, -0.9989853 +0.04503752j,
         0.99853893-0.05403698j, -0.9980115 +0.06303206j],
       [ 1.        -0.j        , -0.99862554-0.05241206j,
         0.99450595+0.10468004j, -0.98765255-0.15666027j,
         0.97808418+0.20820985j, -0.96582714-0.25918708j,
         0.95091512+0.30945183j, -0.93338912-0.35886592j],
       [ 1.        -0.j        , -0.99931104-0.03711385j,
         0.99724512+0.07417657j, -0.99380509-0.11113707j,
         0.98899567+0.14794444j, -0.98282351-0.18454795j,
         0.9752971 +0.22089717j, -0.96642682-0.25694202j],
       [ 1.        +0.j        ,  0.18728352+0.9823059j ,
        -0.92984977+0.36793941j, -0.53557459-0.84448793j,
         0.72924118-0.68425675j,  0.8087243 +0.5881879j ,
        -0.42631971+0.90457255j, -0.96840961-0.24936484j]])